## Dyna planning

#### Background

The aim of this part of the assignment is to introduce you to the concept of Dyna (Sutton, 1990). By now you will have heard about model-free (MF) and model-based (MB) approaches to control. MF algorithms learn and store estimates of the state-action value function. This means that control simply involves retrieving those cached value estimates and choosing among the available actions by comparing their worth. MB control, on the other hand, involves the use of a model to calculate those values. In the RL literature this is typically known as planning. Planning has the advantage of affording behavioural flexibility, since once a change in the world is discovered the model can be updated and the values re-calculated in a way that reflects the global knowledge of the environment. By contrast, MF algorithms need many experiences to propagate the information about the change to other states.

Dyna (Sutton, 1990) is an integrated architecture which combines the merits of MF and MB approaches. Whilst interacting with the environment (i.e., being online), Dyna learns MF state-action values as well as a model of its environment (in the most general sense this involves both the transition and reward models). Whilst not interacting with the environment (i.e., being offline -- this can be in between consecutive moves or episodes, or during the equivalent of sleep in animals), Dyna uses its learnt model to additinally train the MF values. That is, the model now acts as a simulator of the environment and provides additional experiences for learning. This means that at decision time Dyna is fast to react, since it acts according to an MF policy by simply retrieving the relevant values; however, those values have been trained by a model and therefore contain some portion of the global knowledge of the environment collected so far.

One particularly critical aspect of learning is exploration, and in fact Dyna's original motivation was to improve the efficiency of exploration. You are invited to read the original paper by Sutton (1990) to familiarise yourself with the idea and the sort of problems it attempts to solve. The pdf of the paper can be found in the `papers` folder of this repository.

#### Task 1 [10 marks]

The first part of the assignment is to reproduce some of the results from the original Dyna paper (Sutton, 1990). In particular, **your task is to generate and visualise data plotted in figure 6** in that paper. You can neglect Dyna-PI and only implement Dyna-$Q$- and Dyna-$Q$+.

To make your life a little easier, and to let you jump right into the more interesting stuff, you are provided with the environment simulator located in `environment.py`, as well as a blueprint of the main code for the agent. That is, you have access to the file `agent.py` where you will find the `DynaAgent` class. This class has a method called `simulate` with the main simulation loop already implemented.

**Your task is to fill in the missing implementation** in the `agent.py` file. Thus, you are tasked to complete the following functions:
- `_policy`. This is the typical $\pi(a\mid s)$ which specifies how the agent chooses actions in any given state
- `_update_qvals`. This is the $Q$-value update rule
- `_update_experience_buffer`. This updates the agent's experience buffer from which it then samples planning updates
- `_update_action_count`. This counts the number of moves elapsed since each action has last been attempted
- `_plan`. This is the function which lets the agent plan

Once that's done you can run the code below which will hopefully reproduce the figure

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from write_load import load_env
from agent import DynaAgent

In [2]:
# load environments
maze_conf_path = os.path.abspath(os.path.join(os.getcwd(), 'envs'))
maze1_conf = load_env(os.path.join(maze_conf_path, 'dyna1.txt')) # maze with only the right path open 
maze2_conf = load_env(os.path.join(maze_conf_path, 'dyna2.txt')) # maze with both paths open

In [3]:
# initialise the agents
# note that alpha is the learning rate (instead of beta as in the paper)
dyna_qplus  = DynaAgent(alpha=0.5, gamma=0.9, epsilon=0.001)
dyna_qminus = DynaAgent(alpha=0.5, gamma=0.9, epsilon=0)
""" Introduce an epsilon greedy agent. Best action is chosen and sometimes exploration. """
eps_greedy = DynaAgent(alpha=0.5, gamma=0.9, epsilon=-1)

In [4]:
np.random.seed(2)
# run simulations
num_trials = 3000
num_runs   = 50
perf       = [np.zeros((num_runs, num_trials*2)),np.zeros((num_runs, num_trials*2)), np.zeros((num_runs, num_trials*2))]
agents     = [dyna_qplus, dyna_qminus, eps_greedy]

for idx_agent, agent in enumerate(agents):

    for idx_run in range(num_runs):
        agent.init_env(maze1_conf)
        agent.simulate(num_trials=num_trials, reset_agent=True, num_planning_updates=10)
        # world change
        agent.init_env(maze2_conf)
        agent.simulate(num_trials=num_trials, reset_agent=False, num_planning_updates=10)
        # save performance
        perf[idx_agent][idx_run, :] = agent.get_performace()
        if (idx_run+1)%10 == 0:
            print('done with run %u/%u'%(idx_run+1, num_runs))

# average cumulative reward
avg_perf_dyna_qplus  = np.mean(perf[0], axis=0)
avg_perf_dyna_qminus = np.mean(perf[1], axis=0)
avg_perf_eps_greedy = np.mean(perf[2], axis=0)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [5]:
# plot Figure 6 from Sutton (1990) here
plt.figure(figsize=(6, 4))
plt.plot(avg_perf_dyna_qplus,  label='Dyna-Q+')
plt.plot(avg_perf_dyna_qminus, label='Dyna-Q-')
plt.plot(avg_perf_eps_greedy, label='Eps-Greedy')
plt.axvline(3000, linestyle='--', c='r')
plt.legend()

NameError: name 'avg_perf_dyna_qplus' is not defined

<Figure size 600x400 with 0 Axes>

Describe in your own words the apparent differences you observe in the above plot. Some of those differences involve the particular choice of the exploration bonus used in this algorithm. Suggest and implement another sensible exploration bonus and compare the performance of your agent against Dyna-$Q$+ and Dyna-$Q$-. Explain your choice. 

Dyna Q-plus will perform better since a bigger epsilon will increase the exploration. This is especially efficient when the world is changing. Dyna Q- doesn't have any exploration, this means if the world changes the agent won't find the shortcut since it always takes the path which the agent thought was best before.

## 2-step task

#### Background

The 2-step task is one of the most iconic RL tasks (Daw et al. 2011). It was designed to dissect the relative contributions of the MF and MB systems in human choices. There are multiple ways in which MB information can enter choice. For instance, as you will have seen in the case of Dyna, the MF values are additionally trained by the MB system during offline behavioural states. In fact, this process of Dyna-style planning parallels closely hippocampal replay which has been suggested to implement MB planning (Mattar \& Daw, 2018).

For the purpose of this exercise, we will assume that the choice is guided by a linear combination of the MF and MB values. Thus, by tweaking the relative contribution of each, you would expect different behaviours to emerge. Classically, the measure of this balance used in the 2-step task is stay probability. That is, the probability that the subject/agent repeats the same first-stage choice conditioned on the outcome of the second stage in the previous trial.

#### Task 2 [20 marks]

For this part of the assigniment, **your task is to reproduce and visualise data plotted in figure 2** in Daw et al. (2011). There is no pre-implemented code for the agent, except for some basics in the `agent.py` file where you can find the `TwoStepAgent` class. Therefore, you have to follow the methodology in the paper and implement it yourself. You can find the relevant paper in the `papers` folder of this git repository. The only provided code is the one below, as well as the `get_stay_probabilities` method.

In [16]:
from agent import TwoStepAgent

In [18]:
# initialise the agent. The parameters are taken from the paper
agent = TwoStepAgent(alpha1=0.54, alpha2=0.42, beta1=5.19, beta2=3.69, lam=0.57, w=0.39, p=0.11)

In [19]:
# 2-step task
def create_dict():
    """ Define the different states as dictionaries. 
                - name is the name of the state
                - state_num is the number of the state
                - Boolean if there is a reward
                - the possbile actions in the state
                - the name of the actions
                - the beta parameter for the softmax function """
                    
    """ Q-values for the model free, model based and combination of both.
    The Q-values are stored in a 2D array. The first dimension is the state,
    the second the action."""
    Q_MF =np.zeros((3,2))   
    Q_MB =np.zeros((3,2))
    Q_net =np.zeros((3,2))         
                
    stage_c = { 'name': 'stage_c',        'state_num' : 2,                      'reward' : True,                
                'action' : [0, 1],        'action_name' : ['c0', 'c1'],        
                'beta' : 0.2}
    
    stage_b = { 'name': 'stage_b',        'state_num' : 1,                      'reward' : True,               
                'action' : [0, 1],        'action_name' : ['b0', 'b1'],         
                'beta' : 0.2}
    
    stage_a = { 'name': 'stage_a',        'state_num' : 0,                      'reward' : False, 
                'action' : [0, 1],        'action_name' : ['a0', 'a1'],         
                'beta' : 0.2}


    """ Define the connections between the states and action. In stage_a we have 2
        actions which lead to the different stages b and c. Further, the transition     
        probabilities of entering these stages are defined."""
    Connection_dict = { 'stage_a':  {'action' :     {0 : [stage_b , stage_c], 
                                                     1: [stage_b , stage_c]},
                                    'trans_prob' :  {0 : [[0.7, 0.3],[0.3, 0.7]],
                                                     1 : [[0.3, 0.7],[0.7, 0.3]]}},
                                    
                        'stage_b' : {'action' :     {0 : [], 1 : []},
                                     'trans_prob' :  0},
                        'stage_c' : {'action' :     {0 : [], 1 : []},
                                     'trans_prob' :  0}}
        
    return stage_a, stage_b, stage_c, Connection_dict, Q_MF, Q_MB, Q_net

stage_a, stage_b, stage_c, Connection_dict, Q_MF, Q_MB, Q_net = create_dict()

In [ ]:
""" Caluculate the reward. """
def reward(current_state, action):
    """ If there is an reward in the current state. """
    if current_state['reward']:
        
        """ Add gausian Noise to the reward """
        gaussian_noise = np.random.normal(0, 0.025)
        """ Make the reward commulative. """
        current_state['reward_prob'][action] +=  gaussian_noise
        
        """ Create a reflective border at 0.25 and 0.75 for th reward probability """
        if current_state['reward_prob'][action] > 0.75:
            current_state['reward_prob'][action] = 0.75 - (current_state['reward_prob'][action] - 0.75)
        elif current_state['reward_prob'][action] < 0.25:
            current_state['reward_prob'][action] = 0.25 + (0.25 - current_state['reward_prob'][action])
            
        
        """ Get the reward """               
        reward = np.random.choice([1, 0], p = [current_state['reward_prob'][action], 1 - current_state['reward_prob'][action]])
        
        return reward
    
        """ If there is no reward in state return 0. """
    else:
        return 0

In [20]:
""" Update the Q-Model free updates. The update is done according to the Sutton paper."""
def Q_MF_update(Q_MF, current_state, action, next_state, next_action, alpha, lambda_val):
    
    """ If we are not in the first stage"""
    if current_state['state_num'] != 0:
        delta = reward(next_state, next_action) - Q_MF[current_state['state_num'], action]
        Q_MF[current_state['state_num'], action] +=  alpha * delta
                
        """ If we are in the first state"""
    elif current_state['state_num'] == 0:
        delta =  Q_MF[next_state['state_num'], next_action] - Q_MF[current_state['state_num'], action]
        Q_MF[current_state['state_num'], action] +=  alpha * lambda_val * delta
        
        """ If we are in an empty state. """
    else:
        Q_MF[current_state['state_num'], action] = 0
    
    return Q_MF

In [21]:
""" Update the Q-Model free updates. The update is done according to the Sutton paper."""
def Q_MB_update(Q_MB, current_state, action, prob_order):
    
    """ If we are in the first state, include the transition probabilities"""
    if current_state['state_num'] == 0:
        
        P_stage_b = Connection_dict[stage_a['name']]['trans_prob'][action][prob_order]
        P_stage_c = Connection_dict[stage_a['name']]['trans_prob'][action][prob_order]
        
        """ Get the maximum Q-value of the next state and actions. """
        Q_MB[current_state['state_num'], action] = P_stage_b  * np.max(Q_MF[1,0], Q_MF[1,1]) + P_stage_c * np.max(Q_MF[2,0], Q_MF[2,1])
     
        """ If we are not in the second state the Q_MB equals the Q_MF"""   
    elif current_state['state_num'] == 1:
        
        Q_MB[current_state['state_num'], action] = Q_MF[current_state['state_num'], action]

    
    return Q_MB


In [22]:
""" Update the Q_net value. """
def Q_net(Q_net, stage, action, Q_MF, Q_MB, w):
    Q_net[stage['state_num'], action] = w * Q_MF[stage['state_num'], action] + (1-w) * Q_MB[stage['state_num'], action]
    return Q_net

In [23]:
""" Check if the last action is the same as the current action and if we
    we are in the first stage."""
def rep(stage, possible_action, last_action):
    """ Check if any last action was executed. """
    if last_action == -1:
        return 0
    elif (stage['name'] == 'stage_a') and (possible_action == last_action):
       return 1
    else:
        return 0
    

In [29]:
""" Select the actions. """
def select_action(stage, last_action, Q_net, prob_order, p):
    
    p_a_given_s = []
    Nominator, Denominator = 0, 0
    
    """ Use a softmax function to calulate the probabiltiies of the actions by including the Q_net values, p and the rep function. """
    for possible_action in stage['action']:
        Nominator = np.exp(stage['beta'] * Q_net[stage['state_num'], possible_action] + p * rep(stage,possible_action, last_action)) 
        
        Denominator =  sum(np.exp(stage['beta'] * Q_net[stage['state_num'], action] + p * rep(stage,action, last_action)) for action in stage['action']) 
        prob = Nominator/Denominator
        """ Append the probability to the list"""
        p_a_given_s.append(prob)
        
   
    """ Weigh with this the possible action """
    action = np.random.choice(stage['action'], p = p_a_given_s)
    
    """ Get the next stage from the connection_dict, by taking into account the transition probabilities."""
    if Connection_dict[stage['name']]['action'][action] != []:
        next_state = np.random.choice(Connection_dict[stage['name']]['action'][action], p = Connection_dict[stage['name']]['trans_prob'][prob_order][action])
    else:
        next_state = []
    
    
    return action, next_state


In [31]:
""" Look how aften action a and b was choosen and switch transitions probabilities,
    if one action was chosen more often. """
def count_decide_prob(action, next_state):
    
    if (action == 0 and next_state['name'] == 'stage_b') or (action == 1 and next_state['name'] == 'stage_c'):
        counter_1 += 1
    elif (action == 0 and next_state['name'] == 'stage_C') or (action == 1 and next_state['name'] == 'stage_b'):
        counter_2 += 1
        
    if counter_1 > counter_2:
        prob_order = 0
    else:
        prob_order = 1
        
    return prob_order

In [32]:
""" For testing """
agent._init_history()

current_state = stage_a

last_action = -1
prob_order = -1
counter_1, counter_2 = 0, 0

action, next_state = select_action(current_state, last_action, Q_net, prob_order, p = 0.5)
            
prob_order = count_decide_prob(action, next_state)

last_action = action

next_action, _ = select_action(next_state, last_action, Q_net, prob_order, p = 0.5)

r = reward(next_state, next_action)

Q_MF = Q_MF_update(Q_MF, current_state, action, next_state, next_action, alpha = 0.5, lambda_val = 0.5)
Q_MF = Q_MF_update(Q_MF, next_state, next_action, _, _, alpha = 0.5, lambda_val = 0.5)

Q_MB = Q_MB_update(Q_MB, current_state, action, prob_order)
Q_MB = Q_MB_update(Q_MB, next_state, next_action, prob_order)

Q_net = Q_net(Q_net, current_state, action, Q_MF, Q_MB, w = 0.5)
Q_net = Q_net(Q_net, next_state, next_action, Q_MF, Q_MB, w = 0.5)

agent._update_history(next_action, next_state, r)



TypeError: 'function' object is not subscriptable

In [8]:
np.random.seed(2)
# run simulations
num_trials  = 201
num_averg   = 17
stay_probas = np.zeros((num_averg, 4))
for n in range(num_averg):
    agent.simulate(num_trials)
    stay_probas[n, :] = agent.get_stay_probabilities()
    
    """ Initialize the agent. """
    agent._init_history()

    """ Intialize the first state"""
    current_state = stage_a

    """ Initialize the last action, the prob_order and counter"""
    last_action = -1
    prob_order = -1
    counter_1, counter_2 = 0, 0

    """ Chose action and next state"""
    action, next_state = select_action(current_state, last_action, Q_net, prob_order, p = 0.5)
                
    """ Update the transition probabilities order """
    prob_order = count_decide_prob(action, next_state)

    """ Set last action in first stage as action """
    last_action = action

    """ Get the next action and next state. Next state isn't used."""
    next_action, _ = select_action(next_state, last_action, Q_net, prob_order, p = 0.5)

    """ Get reward for second state and second action """
    r = reward(next_state, next_action)

    """ Updatae Q-Model free values. """
    Q_MF = Q_MF_update(Q_MF, current_state, action, next_state, next_action, alpha = 0.5, lambda_val = 0.5)
    Q_MF = Q_MF_update(Q_MF, next_state, next_action, _, _, alpha = 0.5, lambda_val = 0.5)

    """ Updatae Q-Model based values. """
    Q_MB = Q_MB_update(Q_MB, current_state, action, prob_order)
    Q_MB = Q_MB_update(Q_MB, next_state, next_action, prob_order)

    """ Updatae Q-Model net values. """
    Q_net = Q_net(Q_net, current_state, action, Q_MF, Q_MB, w = 0.5)
    Q_net = Q_net(Q_net, next_state, next_action, Q_MF, Q_MB, w = 0.5)

    """ Update the history of the agent. """
    agent._update_history(next_action, next_state, r)
        
        

AttributeError: 'TwoStepAgent' object has no attribute 'history'

In [ ]:
plt.figure(figsize=(6, 4))
plt.bar(1, np.mean(stay_probas[:, 0]), facecolor='blue', label='common')
plt.bar(2, np.mean(stay_probas[:, 1]), facecolor='red', label='rare')
plt.bar(3, np.mean(stay_probas[:, 2]), facecolor='blue')
plt.bar(4, np.mean(stay_probas[:, 3]), facecolor='red')
plt.ylim(0.5, 1)
plt.xticks([1.5, 3.5], ['rewarded', 'unrewarded'])
plt.legend()

Describe in your own words the apparent differences between the MF and MB agents. What do the data plotted with best-fitting parameters tell you about the relative contributions of MF and MB systems to subjects' choices?

The graphic shows that the model free or reinforcement algorithm doesn't take the causal strucutre of the probabilities into account since if there was a wrong prediction this prediction is backwards propagted to the step before. The model-based algorithm creates a causal structure of the envrionemnt. Therefore, the agent takes the probabilities into account and the stay probabilities differ dependent on, if the transition probability is common or rare.